In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.ibmq import *
from qiskit.tools.monitor import job_monitor
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
sim = Aer.get_backend('qasm_simulator')

In [ ]:
def oracleS(s):
    n = len(s)
    qc = QuantumCircuit(2*n)
    for i in range(n):
        if s[i] == '1':
            for j in range(n):
                if s[j] == '1':
                    qc.cx(i,n+j)
    return qc

In [ ]:
def simons(s):
    n = len(s)
    h = QuantumCircuit(2*n,2*n)
    h.h(range(n))
    h.barrier()
    qc = h.compose(oracleS(s))
    qc.barrier()
    r = range(n,2*n)
    qc.measure(r,r)
    qc.barrier()
    qc.compose(h,inplace=True)
    r = range(0,n)
    qc.measure(r,r)
    return qc

In [ ]:
s = "11"
n = len(s)
qc = simons(s)
qc.draw('mpl')

In [ ]:
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= n
                                       and not x.configuration().simulator 
                                       and x.status().operational==True and x.name()!='ibmq_bogota'))
print("least busy backend: ", backend)

In [ ]:
m = execute(qc,backend=sim,shots=1024).result()
print(m)
plot_histogram(
    m.get_counts()
    #execute(qc,backend=sim,shots=1024).result().get_counts()
)

In [ ]:
job = execute(qc,backend=backend,shots=1024)
job_monitor(job)
plot_histogram(
    job.result().get_counts()
)

In [ ]:
meas_fitter = CompleteMeasFitter(cal_results, state_label)
meas_fitter.plot_calibration()

In [ ]:
mit_res = meas_fitter.filter.apply(results)